#**Importe de Librerias**

Para simplificar el calculo y optimizar el código, se ha implementado la clase `SkyCoord` del modulo `astropy` para la representación, manipulación y transformación de coordenadas celestes entre sistemas.

In [1]:
# @title
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from astropy.coordinates import SkyCoord



---


#**Cargue de Archivos**

In [2]:
# @title
CMa_data = pd.read_csv("cma_data.csv")[['ra', 'dec', 'lam']]
data_colors = pd.read_csv('colors.csv')



---


#**Calculo Relativista**

Empleando la clase `SkyCoord` se define el objeto `CMa_stars` que contendrá las coordenadas celestes del conjunto de estrellas provisto, así como `CMa_center` que posee las coordenadas del centro de la constelación *Canis Majoris*

In [3]:
# @title
#Coordenadas Celestes
CMa_center = SkyCoord('06h55m29.2s', '-22d0m34.3s')
CMa_stars = SkyCoord(CMa_data['ra'], CMa_data['dec'], unit = 'deg')

###**1. Colimación**

La cantidad $\theta$ corresponde a la separación angular que se observa entre el centro de la constelación *Canis Majoris* y cada una de las estrellas de la constelación para el marco de referencia en reposo (*Earth's Frame*).

Asumiendo que todas las estrellas se encuentran sobre la superficie de una esfera imaginaria que denominamos *Boveda Celeste*, la separación angular $\theta$ corresponde a la distancia entre dos puntos de la superficie esférica. El calculo de esta distancia sigue las relaciones de la geometría esférica, donde $\alpha$ y $\delta$ hacen referencia a los valores de *Declinación* y *Ascención Recta* tal que

$$\cos\theta=\sin\delta_1\sin\delta_2+\cos\delta_1\delta_2\cos(\alpha_2-\alpha_1)$$

Desde la clase `SkyCoord` recurrimos a la función `self.separation(other)`, donde `self` y `other` son objetos `SkyCoord` que contienen las coordenadas celestes de puntos sobre la *Boveda Celeste*, de modo que podemos calcular sencillamente la separación angular $\theta$ entre el centro de la constelación *Canis Majoris* y las estrellas alrededor.

In [4]:
# @title
#Separación Angular
theta = CMa_center.separation(CMa_stars)

Teniendo distintos valores de la velocidad $v$ a la que se mueve la nave en dirección al centro de la constelación *Canis Majoris*, definimos un array que contiene los factores relativistas $\beta$ y $\gamma$

In [5]:
# @title
#Valores de los Factores Relativistas
beta = np.array([0.1, 0.2, 0.3, 0.4])
gamma = (1 - beta**2)**-0.5

Puesto que el movimiento relativo se da en dirección hacia el centro de la constelación, se satisface la siguiente relación que establece la separación angular entre el centro de la constelación y las estrellas alrededor, observada por el sistema en movimiento.

$$\tan{\theta'}=\frac{\sin\theta}{\gamma(\beta+\cos\theta)}$$

Conocidos los valores de $\theta$ entre el centro de la constelación y cada una de las estrellas de la constelación, podemos obtener sus respectivos valores de $\theta$ que se calculan a continuación.

In [6]:
# @title
#Colimación Relativista
theta_prime = [np.arctan2(np.sin(theta),
                          g*(np.cos(theta) + b))
               for g, b in zip(gamma, beta)]

Ahora que hemos determinado la nueva separación angular $\theta'$ observada por el marco de referencia en movimiento $S'$, debemos determinar las nuevas coordenadas celestes de las estrellas que se han desplazado sobre la geodésica de distancia.

En Astronomía de Posición se define el *Angulo de Posición* $\phi$ como el angulo, con origen en algún punto sobre la *Boveda Celeste*, que se subtiende desde el *Polo Norte Celeste* hacia algún otro punto sobre la *Boveda*.

En particular, la clase `SkyCoord` posee la función `self.position_angle(other)` que calcula dicha cantidad con base en la siguiente relación de la geometría esférica.

$$\tan\phi=\frac{\sin(\alpha_1-\alpha_2)}{\cos\delta_2\tan\delta_1-\sin\delta_2\cos(\alpha_1-\alpha_2)}$$

De esta forma, podemos calcular el *Angulo de Posicion* entre el centro de la constelación *Canis Majoris* y cada una de las estrellas a su alrededor

In [7]:
# @title
phi = CMa_center.position_angle(CMa_stars)

Una de los beneficios que nos ofrece conocer el *Angulo de Posición* es que podemos propagar las coordenadas de un punto sobre la geodésica de distancia entre dos puntos. Con esto nos referimos a que, dado un *Angulo de Posición* $\phi$ y una separación angular $\theta'$, podemos determinar las nuevas coordenadas del punto. En especial, este calculo sigue la proxima relación de la geometría esférica.

$$\cos\delta_2'=\cos\delta_1\cos\theta'+\sin\delta_1\theta'\cos\phi$$

$$\tan(\alpha_2'-\alpha_1)=\frac{\sin\delta_1\sin\theta'\sin\phi}{\cos\theta'-\cos\delta_2'\cos\delta_1}$$

La clase `SkyCoord` posee la función `self.directional_offset_by(phi, theta')` el cual, a partir de las coordenadas del centro de la constelación *Canis Majoris*, junto con los *Angulos de Posición* y desplazamiento angular de *Colimación*, calcula las nuevas coordenadas de las estrellas alrededor en el sistema $S'.

In [ ]:
# @title
CMa_stars_prime = [CMa_center.directional_offset_by(phi, t)
                   for t in theta_prime]

###**2. Efecto Doppler**

El corrimiento en la *Longitud de Onda* de un pulso de luz se afecta unicamente en la dirección paralela al movimiento, de modo que solamente la componente de velocidad paralela produce el desplazamiento de frecuencias.

$$\lambda'=\lambda\gamma(1-\beta\cos\theta)$$

A continuación calculamos las nuevas *Longitudes de Onda* observadas desde el marco de referencia en movimiento $S'$ y asignamos su respectivo color con base en la escala *Hexadecimal* de colores.

In [8]:
# @title
#Corrimiento Doppler
lambda_doppler = [CMa_data.lam*g*(1 - b*np.cos(t))
                   for g, b, t in zip(gamma, beta, theta)]

#Colores en Earth's Frame
colors = [data_colors.hex[data_colors.lamc == i].iloc[0]
          for i in 5*round(CMa_data.lam/5)]

#Nuevos Colores por Efecto Doppler
colors_doppler = [[data_colors.hex[data_colors.lamc == i].iloc[0]
                  for i in 5*round(lambda_/5)]
                  for lambda_ in lambda_doppler]



---

#**Visualización de Resultados**

En la siguiente gráfica podemos apreciar los efectos de *Colimación* y *Corrimiento Doppler Relativista*, donde las estrellas más lejanas presentan estos efectos relativistas de forma más pronunciada que las estrellas un poco más cercanas en la dirección de movimiento.

In [9]:
# @title
fig = go.Figure()

fig.add_trace(go.Scatter(x = [CMa_center.ra.value],
                         y = [CMa_center.dec.value],
                         name = 'CMa Center',
                         marker_symbol = 'x',
                         mode = 'markers',
                         marker_size = 6,
                         marker_color = 'black'))

fig.add_trace(go.Scatter(x = CMa_stars.ra.value,
                         y = CMa_stars.dec.value,
                         marker_color = colors,
                         mode = 'markers',
                         name = "Earth's Frame"))

for i in range(len(gamma)):
  fig.add_trace(go.Scatter(x = CMa_stars_prime[i].ra.value,
                           y = CMa_stars_prime[i].dec.value,
                           marker_color = colors_doppler[i],
                           mode = 'markers',
                           name = f'$\\beta={beta[i]}$'))

fig.update_layout(height = 700, width = 900,
                  xaxis_title = 'RA [deg]',
                  yaxis_title = 'DEC [deg]',
                  title = dict(text = 'Colimación y Efecto Doppler Relativista',
                               x = 0.5, y = 0.96),
                  legend = dict(orientation="h",
                                yanchor="bottom",
                                y=1.01, x = 1,
                                xanchor="right" ))

fig.show()